In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
!pip install gcsfs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import gcsfs
import tensorflow as tf
from tensorflow.python.lib.io import file_io

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# path =r'C:/Users/depri/Desktop/OPS813 cloud computing/Project/case1/amex-nyse-nasdaq-stock-histories/fh_20190217/full_history'
fs = gcsfs.GCSFileSystem(project='ornate-factor-231603')
# allFiles = glob.glob(path + "/*.csv")
allFiles = fs.ls('case2data/full_history')
# allFiles
alldf = []
i=0

for file_ in allFiles:
    i+=1
    if i%100 == 0:
      print(i)
#     if i==400:
#         break
    with fs.open(file_) as f:
      dftemp = pd.read_csv(f,index_col=None, header=0)
#     dftemp = pd.read_csv(file_,index_col=None, header=0)
#     dftemp = df_npd.compute()
    #Figure out a way to invert the data frame
    dftemp = dftemp[::-1]
    #Calculating the 200 day moving average before merging all data
    dftemp["200d"] = np.round(dftemp["adjclose"].rolling(window = 200).mean(), 2)
    #Calculating the 50 day moving average before merging all data
    dftemp["50d"] = np.round(dftemp["adjclose"].rolling(window = 50).mean(), 2)
    #Adding a new colmn stock name to identify the stock in question
    dftemp["StockName"]=os.path.basename(file_).split('.')[0]
    alldf.append(dftemp)

df = pd.concat(alldf, axis = 0, ignore_index = True)
df.columns = ["Date", "Volume","Open", "Close", "High", "Low", "AdjustedClose","200d","50d","StockName"]
# print(df) 

100
200


In [5]:
#Finding stocks in the last month with golden cross and visualizing it
allgoldencrossings = []
alldeathcrossings = []
for key, grp in df.groupby('StockName'): # handling data by each stock.
  previous_50 = grp['50d'].shift(1)             # shifting data by 1.
  previous_200 = grp['200d'].shift(1)           # shifting data by 1.
  deathcrossing = ((grp['50d'] <= grp['200d']) & (previous_50 > previous_200))
  goldencrossing = ((grp['50d'] >= grp['200d']) & (previous_50 < previous_200))
  allgoldencrossings.append(goldencrossing)   # Comparing current value of 50d and 200d and comparing previous value of 50d and 200d to see if the golden cross is achieved.
  # figuring out whether I am having golden cross or not.
  alldeathcrossings.append(deathcrossing)
df['GoldenCross'] = pd.concat(allgoldencrossings)
df['DeathCross'] = pd.concat(alldeathcrossings)
df['WeekChange'] = df['AdjustedClose'].pct_change(periods=7)*100
df['TwoWeekChange'] = df['AdjustedClose'].pct_change(periods=14)*100
df['ThreeWeekChange'] = df['AdjustedClose'].pct_change(periods=21)*100
df['MonthChange'] = df['AdjustedClose'].pct_change(periods=30)*100
df['TwoMonthChange'] = df['AdjustedClose'].pct_change(periods=60)*100
df['ThreeMonthChange'] = df['AdjustedClose'].pct_change(periods=90)*100
df['FourMonthChange'] = df['AdjustedClose'].pct_change(periods=120)*100
df['FiveMonthChange'] = df['AdjustedClose'].pct_change(periods=150)*100
df['SixMonthChange'] = df['AdjustedClose'].pct_change(periods=180)*100

# Count of golden crosses
goldenCount = df[df.GoldenCross].GoldenCross.count()

# Count of death crosses
deathCount = df[df.DeathCross].DeathCross.count()

In [19]:
# Percentage of Golden Cross and Death Cross with 1% change
postiveOnePercentChange = (df[df.GoldenCross & (df.WeekChange > 1)].GoldenCross.count()/goldenCount)*100
negativeOnePercentChange = (df[df.DeathCross & (df.WeekChange < 1)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 1% increase " + str(postiveOnePercentChange))
print("Percentage of Death Cross with 1% decrease " + str(negativeOnePercentChange))

Percentage of Golden Cross with 1% increase 54.790660225442835
Percentage of Death Cross with 1% decrease 71.6923076923077


In [11]:
# Percentage of Golden Cross and Death Cross with 2% change
postiveTwoPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 2)].GoldenCross.count()/goldenCount)*100
negativeTwoPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 2)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 2% increase " + str(postiveTwoPercentChange))
print("Percentage of Death Cross with 2% decrease " + str(negativeTwoPercentChange))

Percentage of Golden Cross with 2% increase 58.212560386473434
Percentage of Death Cross with 2% decrease 78.03846153846153


In [12]:
# Percentage of Golden Cross and Death Cross with 3% change
postiveThreePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 3)].GoldenCross.count()/goldenCount)*100
negativeThreePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 3)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 3% increase " + str(postiveThreePercentChange))
print("Percentage of Death Cross with 3% decrease " + str(negativeThreePercentChange))

Percentage of Golden Cross with 3% increase 61.63446054750402
Percentage of Death Cross with 3% decrease 83.92307692307692


In [13]:
# Percentage of Golden Cross and Death Cross with 4% change
postiveFourPercentChange = (df[df.GoldenCross & (df.MonthChange > 4)].GoldenCross.count()/goldenCount)*100
negativeFourPercentChange = (df[df.DeathCross & (df.MonthChange < 4)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 4% increase " + str(postiveFourPercentChange))
print("Percentage of Death Cross with 4% decrease " + str(negativeFourPercentChange))

Percentage of Golden Cross with 4% increase 67.31078904991948
Percentage of Death Cross with 4% decrease 89.8076923076923


In [14]:
# Percentage of Golden Cross and Death Cross with 5% change
postiveFivePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 5)].GoldenCross.count()/goldenCount)*100
negativeFivePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 5)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 5% increase " + str(postiveFivePercentChange))
print("Percentage of Death Cross with 5% decrease " + str(negativeFivePercentChange))

Percentage of Golden Cross with 5% increase 76.61030595813205
Percentage of Death Cross with 5% decrease 95.92307692307692


In [15]:
# Percentage of Golden Cross and Death Cross with 10% change
postiveTenPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 10)].GoldenCross.count()/goldenCount)*100
negativeTenPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 10)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 10% increase " + str(postiveTenPercentChange))
print("Percentage of Death Cross with 10% decrease " + str(negativeTenPercentChange))

Percentage of Golden Cross with 10% increase 50.48309178743962
Percentage of Death Cross with 10% decrease 95.8076923076923


In [16]:
# Percentage of Golden Cross and Death Cross with 15% change
postiveFifPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 15)].GoldenCross.count()/goldenCount)*100
negativeFifPercentChange = (df[df.DeathCross & (df.FourMonthChange < 15)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 15% increase " + str(postiveFifPercentChange))
print("Percentage of Death Cross with 15% decrease " + str(negativeFifPercentChange))

Percentage of Golden Cross with 15% increase 29.790660225442835
Percentage of Death Cross with 15% decrease 96.57692307692308


In [17]:
# Percentage of Golden Cross and Death Cross with 20% change
postiveTwenPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 20)].GoldenCross.count()/goldenCount)*100
negativeTwenPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 20)].DeathCross.count()/deathCount)*100
print("Percentage of Golden Cross with 20% increase " + str(postiveTwenPercentChange))
print("Percentage of Death Cross with 20% decrease " + str(negativeTwenPercentChange))

Percentage of Golden Cross with 20% increase 16.022544283413847
Percentage of Death Cross with 20% decrease 95.61538461538461
